---

# 빅데이터 분석을 위한 실시간 로지스틱 회귀모형에 관현 연구
## : 베이지안 접근법을 중심으로

---

## 1. 배경

---

---

## 2. 해싱을 이용한 가변수 코딩(Feature Hashing)

## 3. 온라인 최적화 알고리즘

---

### 3.1. 확률적 경사 하강법(Stochastic Gradient Descent)

#### [Stochastic Gradient Descent]

#### [알고리즘]

In [26]:
# simplified version of SGD

alpha = 0.92000539999999997 # best for titanic
w = [0.] * D  # weights
n = np.array([0.] * (D))

f = open(c_fi.file_path)
fn = c_fi.l_header_names

for t, row in enumerate(DictReader(f, fieldnames=fn, delimiter=c_fi.seperator)):  # for titanic(comma seperated)
    if t == 0:
        continue
    del row['PassengerId']
    
    y = 1. if row[c_fi.ylab] == '1' else 0.
    del row[c_fi.ylab]
    
    x = get_x_mmh3(row, D)
    p = get_p(x, w)
    w, n = update_w_withn(w, n, x, p, y, alpha)

    p = get_p(x, w)
f.close()


### 3.2. 추정된 밀도 필터링(Assumed Density Filtering)

---

## 4. 실험

### 4.1. 실험(타이타닉 데이터)

### 4.2. 실험(온라인 광고 데이터)

---

## 5. 맺음말

###### [환경 변수]

In [4]:
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt
import numpy as np
import mmh3
import time
from spooky import hash128, hash64, hash32
from sklearn.metrics import roc_curve, auc
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl

In [6]:
class FileInfo:
    def __init__(self
                , _file_path
                , _f_having_header
                , _l_header_names
                , _seperator
                , _l_skip_columns
                , _ylab):
        self.file_path = _file_path
        self.f_having_header = _f_having_header
        self.l_header_names = _l_header_names
        self.seperator = _seperator
        self.l_skip_columns = _l_skip_columns
        self.ylab = _ylab

In [8]:
c_fi = FileInfo(
                r'C:/My/Playground/Git/2016_Thesis/100_Simulation/data/train.csv' # _file_path
                , True # _f_having_header
                , ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex'
                   , 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin' , 'Embarked'] # _l_header_names
                , ',' # _seperator
                , ['PassengerId']# _l_skip_columns
                , 'Survived'# _ylab
                )

In [14]:
D = 2 ** 20
rand_seed = 1004

In [16]:
# csv_row must be dict
def get_x_mmh3(csv_row, D):
    x = [0]
    for key, value in csv_row.items():
        index = mmh3.hash128(str(key) + str(value), seed=rand_seed, x64arch=True) % D
        x.append(index)
    return x

In [17]:
def get_p(x, w):
    wTx = 0.
    for i in x:  # do wTx
        wTx += w[i] * 1.  # w[i] * x[i], but if i in x we got x[i] = 1.
    return 1. / (1. + exp(-max(min(wTx, 20.), -20.)))  # bounded sigmoid

In [18]:
# w must be numpy ndarray
def get_p_cat(x, w):
    wTx = sum(w[x])
    return 1. / (1. + exp(-max(min(wTx, 20.), -20.)))  # bounded sigmoid

In [21]:
def logloss(p, y):
    p = max(min(p, 1. - 10e-12), 10e-12)
    return -log(p) if y == 1. else -log(1. - p)

In [22]:
def update_w_withn(w, n, x, p, y, alpha):
    for i in x:
        w[i] -= (p - y) * alpha / (sqrt(n[i]) + 1.)
        n[i] += 1.
    return w, n

In [23]:
def update_w(w, n, x, p, y, alpha):
    for i in x:
        w[i] -= (p - y) * alpha
        n[i] += 1.
    return w, n